In [ ]:
from datetime import datetime
import json, sys, time
import python_bitbankcc

In [ ]:
pub = python_bitbankcc.public()

In [ ]:
import os
prv = python_bitbankcc.private(
    os.environ['BITBANK_API_KEY'],
    os.environ['BITBANK_API_SECRET']
)

In [ ]:
WEBHOOK_URL = os.environ['SLACK_WEBHOOK_URL']
CHANNEL_ID = os.environ['SLACK_CHANNEL_ID']

In [ ]:
def send_slack(message, text, color='good'):
    if color == 'good':
        message = f':star-struck: {message}'
    if color == 'danger':
        message = f':rage: {message}'

    import requests
    requests.post(
        WEBHOOK_URL,
        json={
            'channel': CHANNEL_ID,
            'username': 'Trading Notifications',
            'icon_url': 'https://ethereum.org/favicon-32x32.png',
            'text': message,
            'mrkdwn': True,
            'attachments': [{
                'title': '→ Trading Operations',
                'title_link': 'https://jhub.name/user/sheng_wei/lab',
                'text': text,
                'color': color,
            }]
        }
    )

In [ ]:
keys = ('price', 'amount', 'side')
completed = {
    (sys.maxsize, 5, 'buy'): False,
    (-sys.maxsize, 5, 'sell'): False
}

In [ ]:
send_slack(
    f'Order confirmed', 'Power by https://jhub.name/', 'good'
)

In [ ]:
def try_to_trade():
    now_ticker = pub.get_ticker('eth_jpy')
    now_buy_jpy = int(now_ticker.get('sell', f'{sys.maxsize}'))
    now_sell_jpy = int(now_ticker.get('buy', '0'))

    all_completed = True
    for vals in completed:
        if not completed[vals]:
            order = dict((keys[i], vals[i]) for i in range(3))
            if order['side'] == 'buy' and now_buy_jpy < order['price']:
                try:
                    prv.order(
                        pair='eth_jpy', price=str(now_buy_jpy), amount=str(order['amount']), side='buy', order_type='market'
                    )
                    completed[vals] = True
                    send_slack(
                        f'Succeeded to buy {order["amount"]} for {now_buy_jpy} JPY', 'Power by https://jhub.name/', 'good'
                    )
                except Exception:
                    send_slack(
                        f'Failed to buy {order["amount"]} for {now_buy_jpy} JPY', 'Power by https://jhub.name/', 'danger'
                    )
            if order['side'] == 'sell' and now_sell_jpy > order['price']:
                try:
                    prv.order(
                        pair='eth_jpy', price=str(now_sell_jpy), amount=str(order['amount']), side='sell', order_type='market'
                    )
                    completed[vals] = True
                    send_slack(
                        f'Succeeded to sell {order["amount"]} for {now_buy_jpy} JPY', 'Power by https://jhub.name/', 'good'
                    )
                except Exception:
                    send_slack(
                        f'Failed to sell {order["amount"]} for {now_buy_jpy} JPY', 'Power by https://jhub.name/', 'danger'
                    )
        all_completed = all_completed and completed[vals]
    return all_completed

In [ ]:
while not try_to_trade():
    time.sleep(60 * 5)